In [3]:
import pandas as pd

from bs4 import BeautifulSoup

from nltk import word_tokenize

In [4]:
df = pd.read_csv(r"D:\data\openparliament\parliamentary_statements_en.csv")

C:\Users\alecr\Anaconda3\envs\openparliament\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4,5,13,17,22) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
df.drop('Unnamed: 0',axis=1, inplace=True)

In [8]:
df.to_csv(r"D:\data\openparliament\parliamentary_statements_en.csv")

In [7]:
df.head()

,id,document_id,time,h1_en,h2_en,member_id,who_en,content_en,sequence_en,wordcount,...,who_hocid,content_fr,statement_type,written_question,source_id,who_context_en,slug,urlcache,wordcount_en,tokenized_content
0,645328,388,2008-02-14 13:15:00-05,Routine Proceedings,Questions Passed as Orders for Returns,\N,NaN,"<p class=""procedural"" data-HoCid=""906765"">(Ret...",86,2,...,\N,"<p class=""procedural"" data-HoCid=""906765"">(Le ...",NaN,R,p906765,NaN,procedural-3,/debates/2008/2/14/procedural-3/,\N,"['(', 'return', 'tabled', ')']"
1,232373,1878,2001-05-03 13:50:00-04,Government Orders,Federal-Provincial Fiscal Arrangements Act,2611,Ms. Marlene Catterall,"<p>Mr. Speaker, I rise on a point of order. I ...",89,39,...,\N,NaN,NaN,NaN,NaN,NaN,marlene-catterall-1,/debates/2001/5/3/marlene-catterall-1/,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a'..."
2,645329,388,2008-02-14 13:15:00-05,Routine Proceedings,Questions Passed as Orders for Returns,1534,Mr. Tom Lukiwski,"<p data-HoCid=""906766"" data-originallang=""en"">...",87,13,...,78722,"<p data-HoCid=""906766"" data-originallang=""en"">...",interjection,NaN,2324890,Parliamentary Secretary to the Leader of the G...,tom-lukiwski-7,/debates/2008/2/14/tom-lukiwski-7/,\N,"['finally', ',', 'mr.', 'speaker', ',', 'i', '..."
3,291892,1748,2003-03-25 10:05:00-05,NaN,NaN,\N,Some hon. members,<p>Agreed.</p>,1,1,...,\N,NaN,NaN,NaN,NaN,NaN,some-hon-members-1,/debates/2003/3/25/some-hon-members-1/,\N,"['agreed', '.']"
4,232374,1878,2001-05-03 13:50:00-04,Government Orders,Federal-Provincial Fiscal Arrangements Act,\N,The Acting Speaker (Mr. Bélair),<p>Is that agreed?</p>,90,3,...,\N,NaN,NaN,NaN,NaN,NaN,the-acting-speaker-mr-belair-9,/debates/2001/5/3/the-acting-speaker-mr-belair-9/,\N,"['is', 'that', 'agreed', '?']"


In [13]:
c = df.content_en[10]

In [14]:
c

'<p>I thank the hon. member.</p>'

In [20]:
soop = BeautifulSoup(c)

In [21]:
soop.text

'I thank the hon. member.'

In [31]:
def tokenize_column (row):
    soup = BeautifulSoup(row)
    text = soup.text.lower()
    words = word_tokenize(text)
    return words

In [32]:
tokenize_column(df.content_en[10])

['i', 'thank', 'the', 'hon', '.', 'member', '.']

In [33]:
df["tokenized_content"] = df.content_en.apply(tokenize_column)

In [34]:
df.to_csv(r"D:\data\openparliament\parliamentary_statements_en.csv")

In [35]:
df.tokenized_content.head()

0                               [(, return, tabled, )]
1    [mr., speaker, ,, i, rise, on, a, point, of, o...
2    [finally, ,, mr., speaker, ,, i, ask, that, al...
3                                          [agreed, .]
4                                [is, that, agreed, ?]
Name: tokenized_content, dtype: object

In [36]:
from nltk.corpus import stopwords

In [38]:
stop_words = set(stopwords.words('english'))

In [40]:
tokens = df.tokenized_content[2]

In [43]:
def remove_stopwords(word_tokens):
    words = [w for w in word_tokens if not w in stop_words]
    return words

In [44]:
remove_stopwords(word_tokens=tokens)

['finally',
 ',',
 'mr.',
 'speaker',
 ',',
 'ask',
 'remaining',
 'questions',
 'allowed',
 'stand',
 '.']

In [45]:
tokens

['finally',
 ',',
 'mr.',
 'speaker',
 ',',
 'i',
 'ask',
 'that',
 'all',
 'remaining',
 'questions',
 'be',
 'allowed',
 'to',
 'stand',
 '.']

'\\N' is the character used in columns that contain nothing else, so removing that character removes nulls

In [13]:
df = df[df.member_id!= '\\N']

In [21]:
import numpy as np

In [37]:
#df.pivot_table(index=['member_id'], columns=['wordcount'], aggfunc=np.sum)
newdf = pd.pivot_table(df, values=['wordcount'], index=['member_id', 'who_en'], aggfunc = 'mean')

In [38]:
newdf.head()

wordcount
member_id who_en                                                        
1000      Mrs. Bonnie Crombie                                  41.715225
          Mrs. Bonnie Crombie (Mississauga—Streetsville, ...  236.659193
          Ms. Bonnie Crombie                                    6.000000
1007      M. Don Davies (Vancouver Kingsway, NPD)             850.000000
          Mr. Don Davies                                       89.085861

In [50]:
newdf.sort_values(by='wordcount', ascending=False)[:10]

,,wordcount
member_id,who_en,
3687,Hon. Mauril Bélanger (for the President of the Treasury Board),2975.000000
3172,Hon. Allan Rock (for the President of the Treasury Board),2855.000000
4206,"Hon. Jody Wilson-Raybould (Minister of Justice and Attorney General of Canada, Lib.)",2621.000000
1267,Hon. Chuck Strahl (Minister of Agriculture and Agri-Food),2566.333333
1279,"Mr. Gurmant Grewal (Newton—North Delta, CPC),",2532.000000
3139,"Hon. Ethel Blondin-Andrew (Minister of State (Children and Youth), Lib.)",2510.333333
929,Hon. Leona Aglukkaq (Minister of Health and Minister of the Canadian Northern Economic Development Agency),2304.000000
3320,Right Hon. Jean Chrétien (for the Minister of State and Leader of the Government in the House of Commons),2252.000000
1762,"Mr. Michel Guimond (Beauport—Montmorency—Côte-de-Beaupré Île-d'Orléans, BQ)",2229.000000


This has multiple different names for member_ids, since if someone is the treasury minister, they get an entry for whether they're in that capacity or as a regular MP. In any case, you can see the top ten in loquaciousness. 

In [42]:
type(newdf.columns)

pandas.core.indexes.base.Index

KeyError: 1000

In [56]:
newdf.iloc[3]

wordcount    850.0
Name: (1007, M. Don Davies (Vancouver Kingsway, NPD)), dtype: float64